## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'gmaps'

In [5]:
# Files to load
weather_data_to_load = "/Users/katiebernstein/Documents/Data Bootcamp/Git/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv"


# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(weather_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fare,PF,-16.7000,-151.0167,77.56,71,94,24.23,overcast clouds
1,1,Geraldton,AU,-28.7667,114.6000,64.81,77,100,8.05,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,32.11,93,100,2.30,overcast clouds
3,3,Veraval,IN,20.9000,70.3667,83.39,80,99,9.08,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,31.66,80,0,1.14,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fare,PF,-16.7000,-151.0167,77.56,71,94,24.23,overcast clouds
3,3,Veraval,IN,20.9000,70.3667,83.39,80,99,9.08,overcast clouds
8,8,Pocone,BR,-16.2567,-56.6228,77.47,32,20,6.08,few clouds
11,11,Pokhara,NP,28.2333,83.9833,80.28,75,100,2.66,overcast clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.04,76,26,19.84,scattered clouds
19,19,Qandala,SO,11.4720,49.8728,88.45,49,7,7.23,clear sky
20,20,Vostok,RU,46.4856,135.8833,80.19,52,98,4.56,overcast clouds
24,24,Urucui,BR,-7.2294,-44.5561,77.58,31,5,7.31,clear sky
32,32,Bathsheba,BB,13.2167,-59.5167,79.41,98,75,20.71,broken clouds
33,33,Hilo,US,19.7297,-155.0900,82.27,83,40,8.05,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Fare,PF,77.56,overcast clouds,-16.7000,-151.0167,
3,Veraval,IN,83.39,overcast clouds,20.9000,70.3667,
8,Pocone,BR,77.47,few clouds,-16.2567,-56.6228,
11,Pokhara,NP,80.28,overcast clouds,28.2333,83.9833,
13,Atuona,PF,77.04,scattered clouds,-9.8000,-139.0333,
19,Qandala,SO,88.45,clear sky,11.4720,49.8728,
20,Vostok,RU,80.19,overcast clouds,46.4856,135.8833,
24,Urucui,BR,77.58,clear sky,-7.2294,-44.5561,
32,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,
33,Hilo,US,82.27,scattered clouds,19.7297,-155.0900,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")    
        

NameError: name 'g_key' is not defined

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/katiebernstein/Documents/Data Bootcamp/Git/World_Weather_Analysis/Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))